# The 20 newsgroups topic analysis

Instead of repeating the IMDB sentiment analysis from the lesson (because frankly, I'm a little bored with sentiment analysis), I will attempt to apply a similar approach to deep-learning NLP classification to a dataset a coworker has recently been messing around with in `scikit-learn`: `sklearn.datasets.fetch_20newsgroups`.

http://people.csail.mit.edu/jrennie/20Newsgroups/

## Setup data

In [1]:
import os
current_dir = os.getcwd()

LESSON_HOME_DIR = current_dir + '/'
DATA_HOME_DIR = LESSON_HOME_DIR + 'data/'

DATASET_DIR = DATA_HOME_DIR + '20_newsgroup/'
MODEL_DIR = DATASET_DIR + 'models/'

In [2]:
if not os.path.exists(MODEL_DIR):
    os.mkdir(DATASET_DIR)
    os.mkdir(MODEL_DIR)

In [3]:
from sklearn.datasets import fetch_20newsgroups

category_subset = [
    'alt.atheism',
    'comp.graphics',
    'comp.os.ms-windows.misc',
    'soc.religion.christian',
]

newsgroups = fetch_20newsgroups(
    subset = 'all',
    categories = category_subset,
    shuffle = True,
    remove = ('headers', 'footers', 'quotes'))

In [4]:
newsgroups.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'soc.religion.christian']

`target_names` are as requested

In [5]:
newsgroups.filenames.shape, newsgroups.target.shape, len(newsgroups.data)

((3754,), (3754,), 3754)

Keras implements `get_word_index()` for the IMDB dataset, which returns an dictionary of word->index derived from a json file hosted on Amazon S3.

It seems bizarre to me to host this when you can easily create it on-demand... anyway, sklearn doesn't provide this. So let's create our own index with `keras.preprocessing.text.Tokenizer` (https://keras.io/preprocessing/text/).

In [6]:
import keras.preprocessing.text
import string

# Workaround to add "Unicode support for keras.preprocessing.text"
# (https://github.com/fchollet/keras/issues/1072#issuecomment-295470970)
def text_to_word_sequence(text,
                          filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                          lower=True, split=" "):
    if lower: text = text.lower()
    if type(text) == unicode:
        translate_table = {ord(c): ord(t) for c,t in zip(filters, split*len(filters)) }
    else:
        translate_table = string.maketrans(filters, split * len(filters))
    text = text.translate(translate_table)
    seq = text.split(split)
    return [i for i in seq if i]
    
keras.preprocessing.text.text_to_word_sequence = text_to_word_sequence

Using Theano backend.
Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [7]:
from keras.preprocessing.text import Tokenizer

vocab_size = 10000

tokenizer = Tokenizer(nb_words=vocab_size)
tokenizer.fit_on_texts(newsgroups.data) # builds the word index
sequences = tokenizer.texts_to_sequences(newsgroups.data)

In [8]:
word_index = tokenizer.word_index

In [9]:
len(word_index)

72905

Reverse the `word_index` with `idx2word`.

In [10]:
idx2word = {v: k for k, v in word_index.iteritems()}

Let's take a look at the first review, both as a list of indices and as text reconstructed from the indices.

In [11]:
', '.join(map(str, sequences[0]))

'24, 2, 60, 566, 52, 20, 4829, 3, 389, 2, 3000, 1339, 2, 155, 386, 84, 901, 76, 4, 115, 24, 2, 88, 566, 76, 92, 402, 3, 525, 101, 2, 155, 385, 6, 1685, 2454, 236, 93, 182, 118, 1350, 335, 7, 108, 5725, 2764, 6, 3000, 8981, 20, 396, 3, 118, 127, 2454, 6, 158, 182, 90, 4499, 14, 129, 11, 2274, 81, 4808, 234, 219, 92, 23, 5605, 6, 720, 10, 3000, 61, 783, 5464, 8, 5725, 7, 1736, 3, 239, 3, 1464, 55, 2, 579, 5464, 214, 701, 45, 91, 23, 2, 1802, 4, 5, 1464, 10, 2, 2694, 74, 318, 129, 386, 436, 5, 24, 11, 30, 3, 102, 1641, 11, 114, 332, 8, 2, 1598, 5, 858, 537, 9, 11, 1237, 96, 386, 95, 17, 23, 3, 952, 26, 8, 9, 121, 23, 219, 9951, 9845, 61, 2104, 95, 96, 5725, 11, 8, 5, 2828, 3, 2501, 197, 4, 127, 863, 720, 200, 102, 337, 127, 1576, 1021, 93, 15, 495, 2, 2157, 4, 94, 396, 3, 5777, 127, 9605, 720, 182, 118, 614, 2221, 60, 261, 3, 583, 8, 9, 135, 2, 154, 1464, 8, 210, 1464'

In [12]:
idx2word[24]

u'on'

In [13]:
' '.join([idx2word[o] for o in sequences[0]])

u"on the one hand there are advantages to having the liturgy stay the same john has described some of these on the other hand some people seem to start out the same old and pay attention better when things get changed around i think innovative priests and liturgy committees are trying to get our attention and make things more meaningful for us it drives me crazy too different people have preferences and needs in liturgy my local parish is innovative i prefer to go to mass at the next parish over sometimes we don't have the option of a mass in the style which best us john put a on it but to just offer it up probably is the solution a related issue that it sounds like john does not have to deal with is that may have different liturgical tastes my husband does like innovative it is a challenge to meet both of our spiritual needs without just going our separate ways when you include the factor of also trying to satisfy our children's needs things get pretty complicated one thing to remembe

In [14]:
newsgroups.target[0], newsgroups.target_names[newsgroups.target[0]]

(3, 'soc.religion.christian')

Distribution of the lengths of sentences:

In [15]:
import numpy as np

lens = np.array(map(len, newsgroups.data))
(lens.max(), lens.min(), lens.mean())

(158791, 0, 1493.157432072456)

Weird that there are sentences with 0 sequences (words) in them...

In [16]:
# get indices of arrays that do NOT satisfy np.nonzero
nonzero_indices = np.unique(np.nonzero(sequences)[0])
zero_indices = set(range(len(sequences))).difference(nonzero_indices)
len(zero_indices)

101

So there are 101 sentences with no words. E.g.

In [17]:
sequences[64], newsgroups.target_names[newsgroups.target[64]]

([], 'alt.atheism')

...sure.

Pad (with zero) or truncate each sentence to make consistent length.

In [18]:
from keras.preprocessing import sequence

seq_len = 1500

data = sequence.pad_sequences(sequences, maxlen=seq_len, value=0)

In [19]:
data[:10]

array([[   0,    0,    0, ...,    8,  210, 1464],
       [   0,    0,    0, ..., 3162,    8,   11],
       [   0,    0,    0, ...,    2,  318, 1142],
       ..., 
       [   0,    0,    0, ...,   47,    7,  740],
       [   0,    0,    0, ..., 2565,  356,  129],
       [   0,    0,    0, ..., 4386,  364, 8254]], dtype=int32)

Finally, let's turn the labels into categorical information.

In [20]:
from keras.utils.np_utils import to_categorical

newsgroups.target = to_categorical(np.asarray(newsgroups.target))

In [21]:
data.shape, newsgroups.target.shape

((3754, 1500), (3754, 4))

Split data into train-test.

In [22]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data, newsgroups.target, test_size=0.33)

In [23]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((2515, 1500), (1239, 1500), (2515, 4), (1239, 4))

## Create simple models

### Single hidden layer NN

The simplest model that tends to give reasonable results is a single hidden layer net. So let's try that. Note that we can't expect to get any useful results by feeding word ids directly into a neural net - so instead we use an embedding to replace them with a vector of 32 (initially random) floats for each word in the vocab.

In [24]:
vocab_size, seq_len

(10000, 1500)

In [25]:
from keras.models import Sequential
from keras.layers import Embedding
from keras.layers.core import Flatten, Dense, Dropout
from keras.optimizers import Adam

# input_length => 1500-word reviews, 32 floats per word
model = Sequential([
    Embedding(vocab_size, 32, input_length=seq_len),
    Flatten(),
    Dense(100, activation='relu'),
    Dropout(0.7),
    Dense(len(newsgroups.target_names), activation='softmax')])

In [26]:
model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_1 (Embedding)          (None, 1500, 32)      320000      embedding_input_1[0][0]          
____________________________________________________________________________________________________
flatten_1 (Flatten)              (None, 48000)         0           embedding_1[0][0]                
____________________________________________________________________________________________________
dense_1 (Dense)                  (None, 100)           4800100     flatten_1[0][0]                  
____________________________________________________________________________________________________
dropout_1 (Dropout)              (None, 100)           0           dense_1[0][0]                    
___________________________________________________________________________________________

In [27]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), nb_epoch=10, batch_size=64)

Train on 2515 samples, validate on 1239 samples
Epoch 1/10
2515/2515 [==============================] - 0s - loss: 1.4609 - acc: 0.2748 - val_loss: 1.3608 - val_acc: 0.3479
Epoch 2/10
2515/2515 [==============================] - 0s - loss: 1.3311 - acc: 0.2938 - val_loss: 1.2943 - val_acc: 0.3850
Epoch 3/10
2515/2515 [==============================] - 0s - loss: 1.2346 - acc: 0.3932 - val_loss: 1.1203 - val_acc: 0.4923
Epoch 4/10
2515/2515 [==============================] - 0s - loss: 1.0246 - acc: 0.5014 - val_loss: 0.8777 - val_acc: 0.5472
Epoch 5/10
2515/2515 [==============================] - 0s - loss: 0.7436 - acc: 0.6433 - val_loss: 0.7756 - val_acc: 0.6077
Epoch 6/10
2515/2515 [==============================] - 0s - loss: 0.5509 - acc: 0.7761 - val_loss: 0.7083 - val_acc: 0.6594
Epoch 7/10
2515/2515 [==============================] - 0s - loss: 0.3676 - acc: 0.8771 - val_loss: 0.6639 - val_acc: 0.6917
Epoch 8/10
2515/2515 [==============================] - 0s - loss: 0.2500 - a

Is a mid-70s validation accuracy.. Good? Bad?

Here are some accuracies [from an official `sklearn` example](http://scikit-learn.org/stable/auto_examples/text/document_classification_20newsgroups.html) that classifies documents by topics using a bag-of-words approach:

```
[('RidgeClassifier', 0.89726533628972649),
 ('Perceptron', 0.88543976348854403),
 ('PassiveAggressiveClassifier', 0.90613451589061345),
 ('KNeighborsClassifier', 0.85809312638580926),
 ('RandomForestClassifier', 0.83813747228381374),
 ('LinearSVC', 0.90022172949002222),
 ('SGDClassifier', 0.90096082779009612),
 ('LinearSVC', 0.87287509238728755),
 ('SGDClassifier', 0.88543976348854403),
 ('SGDClassifier', 0.89874353288987441),
 ('NearestCentroid', 0.85513673318551364),
 ('MultinomialNB', 0.90022172949002222),
 ('BernoulliNB', 0.88396156688839611),
 ('Pipeline', 0.8810051736881005)]
 
 mean: 0.88311688311688319
 ```

So, not a good result in comparison with much simpler approaches. Training accuracy is high, but testing accuracy is much poorer.

As a sanity check, I also ran code from [`pretrained_word_embeddings.py`](https://github.com/fchollet/keras/blob/master/examples/pretrained_word_embeddings.py) (from Keras's examples repository) which also runs against `20_newsgroups` (not the `sklearn` version though), and it was able to achieve:

    loss: 0.3784 - acc: 0.8734 - val_loss: 0.9177 - val_acc: 0.7257
after 10 epochs - again, not as accurate as the 'shallow', bag-of-words models - but comparable to the results I'm receiving here.

### Single conv layer with max pooling

A CNN is likely to work better, since it's designed to take advantage of ordered data. We'll need to use a 1D CNN, since a sequence of words is 1D.

In [28]:
from keras.layers.convolutional import Convolution1D, MaxPooling1D

conv1 = Sequential([
    Embedding(vocab_size, 100, input_length=seq_len, dropout=0.2),
    Dropout(0.4),
    Convolution1D(128, 5, activation='relu'),
    Dropout(0.4),
    MaxPooling1D(5),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.7),
    Dense(len(newsgroups.target_names), activation='softmax')])

In [29]:
from keras.optimizers import RMSprop

conv1.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])
conv1.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_2 (Embedding)          (None, 1500, 100)     1000000     embedding_input_2[0][0]          
____________________________________________________________________________________________________
dropout_2 (Dropout)              (None, 1500, 100)     0           embedding_2[0][0]                
____________________________________________________________________________________________________
convolution1d_1 (Convolution1D)  (None, 1496, 128)     64128       dropout_2[0][0]                  
____________________________________________________________________________________________________
dropout_3 (Dropout)              (None, 1496, 128)     0           convolution1d_1[0][0]            
___________________________________________________________________________________________

In [30]:
conv1.optimizer.lr.get_value().item()

0.0010000000474974513

In [31]:
conv1.fit(X_train, y_train, validation_data=(X_test, y_test), nb_epoch=10, batch_size=64)

Train on 2515 samples, validate on 1239 samples
Epoch 1/10
2515/2515 [==============================] - 7s - loss: 1.4298 - acc: 0.3241 - val_loss: 1.3593 - val_acc: 0.3769
Epoch 2/10
2515/2515 [==============================] - 7s - loss: 1.3111 - acc: 0.3559 - val_loss: 1.2686 - val_acc: 0.4044
Epoch 3/10
2515/2515 [==============================] - 7s - loss: 1.2105 - acc: 0.4052 - val_loss: 1.1784 - val_acc: 0.4439
Epoch 4/10
2515/2515 [==============================] - 7s - loss: 1.0240 - acc: 0.4899 - val_loss: 0.9215 - val_acc: 0.5504
Epoch 5/10
2515/2515 [==============================] - 7s - loss: 0.8506 - acc: 0.5610 - val_loss: 0.7941 - val_acc: 0.6320
Epoch 6/10
2515/2515 [==============================] - 7s - loss: 0.6949 - acc: 0.6636 - val_loss: 0.6857 - val_acc: 0.6820
Epoch 7/10
2515/2515 [==============================] - 7s - loss: 0.5433 - acc: 0.7654 - val_loss: 0.6388 - val_acc: 0.7086
Epoch 8/10
2515/2515 [==============================] - 7s - loss: 0.4313 - a

In [32]:
conv1.optimizer.lr=0.01

In [33]:
conv1.fit(X_train, y_train, validation_data=(X_test, y_test), nb_epoch=4, batch_size=64)

Train on 2515 samples, validate on 1239 samples
Epoch 1/4
2515/2515 [==============================] - 7s - loss: 0.2550 - acc: 0.9082 - val_loss: 0.5456 - val_acc: 0.7837
Epoch 2/4
2515/2515 [==============================] - 7s - loss: 0.2131 - acc: 0.9161 - val_loss: 0.5704 - val_acc: 0.7877
Epoch 3/4
2515/2515 [==============================] - 7s - loss: 0.1961 - acc: 0.9241 - val_loss: 0.6248 - val_acc: 0.7740
Epoch 4/4
2515/2515 [==============================] - 7s - loss: 0.2053 - acc: 0.9264 - val_loss: 0.5908 - val_acc: 0.7797


In [34]:
conv1.fit(X_train, y_train, validation_data=(X_test, y_test), nb_epoch=1, batch_size=64)

Train on 2515 samples, validate on 1239 samples
Epoch 1/1
2515/2515 [==============================] - 7s - loss: 0.1649 - acc: 0.9308 - val_loss: 0.6304 - val_acc: 0.7780


A good improvement over the previous model.

## Pre-trained vectors

You may want to look at wordvectors.ipynb before moving on.

In this section, we replicate the previous CNN, but using pre-trained embeddings.

In [35]:
from keras.utils.data_utils import get_file

def get_glove_dataset(dataset):
    """Download the requested glove dataset from files.fast.ai
    and return a location that can be passed to load_vectors.
    """
    # see wordvectors.ipynb for info on how these files were
    # generated from the original glove data.
    md5sums = {'6B.50d': '8e1557d1228decbda7db6dfd81cd9909',
               '6B.100d': 'c92dbbeacde2b0384a43014885a60b2c',
               '6B.200d': 'af271b46c04b0b2e41a84d8cd806178d',
               '6B.300d': '30290210376887dcc6d0a5a6374d8255'}
    glove_path = os.path.abspath('data/glove/results')
    %mkdir -p $glove_path
    return get_file(dataset,
                    'http://files.fast.ai/models/glove/' + dataset + '.tgz',
                    cache_subdir=glove_path,
                    md5_hash=md5sums.get(dataset, None),
                    untar=True)

In [36]:
from utils import load_array
import pickle

def load_vectors(loc):
    return (load_array(loc+'.dat'),
        pickle.load(open(loc+'_words.pkl','rb')),
        pickle.load(open(loc+'_idx.pkl','rb')))

In [37]:
vecs, words, wordidx = load_vectors(get_glove_dataset('6B.100d'))

Untaring file...


In [38]:
len(wordidx)

400000

The glove word ids and imdb word ids use different indexes. So we create a simple function that creates an embedding matrix using the indexes from imdb, and the embeddings from glove (where they exist).

In [39]:
import re
from numpy.random import normal

def create_emb():
    n_fact = vecs.shape[1]
    emb = np.zeros((vocab_size, n_fact))

    for i in range(1,len(emb)):
        word = idx2word[i]
        if word and re.match(r"^[a-zA-Z0-9\-]*$", word) and word in wordidx:
            src_idx = wordidx[word]
            emb[i] = vecs[src_idx]
        else:
            # If we can't find the word in glove, randomly initialize
            emb[i] = normal(scale=0.6, size=(n_fact,))

    # This is our "rare word" id - we want to randomly initialize
    emb[-1] = normal(scale=0.6, size=(n_fact,))
    emb/=3
    return emb

In [40]:
emb = create_emb()

In [41]:
emb.shape

(10000, 100)

In [42]:
emb_model = Sequential([
    Embedding(vocab_size, 100, input_length=seq_len, dropout=0.2, 
              weights=[emb], trainable=False),
    Dropout(0.25),
    Convolution1D(128, 5, border_mode='same', activation='relu'),
    Dropout(0.25),
    MaxPooling1D(),
    Flatten(),
    Dense(100, activation='relu'),
    Dropout(0.7),
    Dense(4, activation='softmax')])

_Note_: I started seeing lines like `4s - loss: nan - acc: 0.6783 - val_loss: nan - val_acc: 0.2131` where in the previous epoch, `val_acc` was twice that amount. A [quick search on the forums](http://forums.fast.ai/t/why-are-my-losses-nan/2931/2) surfaced this explanation:

    "There is one thing that doesn't look quite right: the final activation is not compatible with that loss function. Categorical cross-entropy expects a 'softmax' activation in the final layer, not 'sigmoid'. Consider changing that to see what happens."
    
**Categorical cross-entropy expects a `softmax` activation in the final layer, not `sigmoid`.** So I switched to `softmax`... I don't recall ever learning this information, however. Should ponder why.

In [43]:
emb_model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])
emb_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_3 (Embedding)          (None, 1500, 100)     0           embedding_input_3[0][0]          
____________________________________________________________________________________________________
dropout_5 (Dropout)              (None, 1500, 100)     0           embedding_3[0][0]                
____________________________________________________________________________________________________
convolution1d_2 (Convolution1D)  (None, 1500, 128)     64128       dropout_5[0][0]                  
____________________________________________________________________________________________________
dropout_6 (Dropout)              (None, 1500, 128)     0           convolution1d_2[0][0]            
___________________________________________________________________________________________

In [44]:
emb_model.fit(X_train, y_train, validation_data=(X_test, y_test), nb_epoch=10, batch_size=64)

Train on 2515 samples, validate on 1239 samples
Epoch 1/10
2515/2515 [==============================] - 7s - loss: 1.4153 - acc: 0.3245 - val_loss: 1.2945 - val_acc: 0.4213
Epoch 2/10
2515/2515 [==============================] - 7s - loss: 1.1306 - acc: 0.4453 - val_loss: 0.9327 - val_acc: 0.5351
Epoch 3/10
2515/2515 [==============================] - 7s - loss: 0.9492 - acc: 0.5245 - val_loss: 0.8429 - val_acc: 0.5537
Epoch 4/10
2515/2515 [==============================] - 7s - loss: 0.8615 - acc: 0.5519 - val_loss: 0.7902 - val_acc: 0.5738
Epoch 5/10
2515/2515 [==============================] - 7s - loss: 0.8365 - acc: 0.5932 - val_loss: 0.7666 - val_acc: 0.6360
Epoch 6/10
2515/2515 [==============================] - 7s - loss: 0.7674 - acc: 0.6445 - val_loss: 0.7028 - val_acc: 0.6691
Epoch 7/10
2515/2515 [==============================] - 7s - loss: 0.7152 - acc: 0.6795 - val_loss: 0.6683 - val_acc: 0.6965
Epoch 8/10
2515/2515 [==============================] - 7s - loss: 0.6710 - a

Let's fine-tune the embedding weights - especially since the words we couldn't find in glove just have random embeddings.

In [45]:
emb_model.layers[0].trainable=True

In [46]:
emb_model.optimizer.lr=1e-4

In [47]:
emb_model.fit(X_train, y_train, validation_data=(X_test, y_test), nb_epoch=10, batch_size=64)

Train on 2515 samples, validate on 1239 samples
Epoch 1/10
2515/2515 [==============================] - 7s - loss: 0.5923 - acc: 0.7431 - val_loss: 0.6339 - val_acc: 0.7207
Epoch 2/10
2515/2515 [==============================] - 7s - loss: 0.5529 - acc: 0.7730 - val_loss: 0.6349 - val_acc: 0.7127
Epoch 3/10
2515/2515 [==============================] - 7s - loss: 0.5522 - acc: 0.7761 - val_loss: 0.6269 - val_acc: 0.7111
Epoch 4/10
2515/2515 [==============================] - 7s - loss: 0.5249 - acc: 0.7861 - val_loss: 0.6437 - val_acc: 0.7103
Epoch 5/10
2515/2515 [==============================] - 7s - loss: 0.4881 - acc: 0.8004 - val_loss: 0.6299 - val_acc: 0.7119
Epoch 6/10
2515/2515 [==============================] - 7s - loss: 0.4582 - acc: 0.8135 - val_loss: 0.6220 - val_acc: 0.7167
Epoch 7/10
2515/2515 [==============================] - 7s - loss: 0.4148 - acc: 0.8330 - val_loss: 0.6290 - val_acc: 0.7256
Epoch 8/10
2515/2515 [==============================] - 7s - loss: 0.4407 - a

In [48]:
emb_model.layers[0].trainable=False

In [49]:
emb_model.optimizer.lr=1e-2

In [50]:
emb_model.fit(X_train, y_train, validation_data=(X_test, y_test), nb_epoch=5, batch_size=64)

Train on 2515 samples, validate on 1239 samples
Epoch 1/5
2515/2515 [==============================] - 7s - loss: 0.3683 - acc: 0.8604 - val_loss: 0.6309 - val_acc: 0.7207
Epoch 2/5
2515/2515 [==============================] - 7s - loss: 0.3784 - acc: 0.8561 - val_loss: 0.6264 - val_acc: 0.7312
Epoch 3/5
2515/2515 [==============================] - 7s - loss: 0.3676 - acc: 0.8640 - val_loss: 0.6252 - val_acc: 0.7288
Epoch 4/5
2515/2515 [==============================] - 7s - loss: 0.3395 - acc: 0.8708 - val_loss: 0.6466 - val_acc: 0.7248
Epoch 5/5
2515/2515 [==============================] - 7s - loss: 0.3268 - acc: 0.8751 - val_loss: 0.6619 - val_acc: 0.7191


Interestingly, the pretrained embeddings didn't provide any improvement...

## Pre-trained vectors + BatchNorm

In the lesson 4 wiki notes, the following advice was given:

> As discussed last lesson, it's good practice to always normalize our inputs. Instead of manually calculating averages and variances across the entire training set, a much simply approach is to start your Keras model with a Batch Normalization layer. This will handle the input normalization for you, and is a good default starting point.

So let's use the previous pre-trained architecture with one slight modification: swapping out the first `Dropout` layer with a `BatchNormalization` layer instead, after the input layer as suggested.

In [51]:
emb = create_emb()

In [52]:
from keras.layers.normalization import BatchNormalization

batch_model = Sequential([
    Embedding(vocab_size, 100, input_length=seq_len, dropout=0.2, 
              weights=[emb], trainable=False),
    Dropout(0.25),
    Convolution1D(128, 5, border_mode='same', activation='relu'),
    Dropout(0.25),
    MaxPooling1D(),
    Flatten(),
    Dense(100, activation='relu'),
    BatchNormalization(axis=1),
    Dropout(0.7),
    Dense(4, activation='softmax')])

In [53]:
batch_model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])
batch_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_4 (Embedding)          (None, 1500, 100)     0           embedding_input_4[0][0]          
____________________________________________________________________________________________________
dropout_8 (Dropout)              (None, 1500, 100)     0           embedding_4[0][0]                
____________________________________________________________________________________________________
convolution1d_3 (Convolution1D)  (None, 1500, 128)     64128       dropout_8[0][0]                  
____________________________________________________________________________________________________
dropout_9 (Dropout)              (None, 1500, 128)     0           convolution1d_3[0][0]            
___________________________________________________________________________________________

In [54]:
batch_model.fit(X_train, y_train, validation_data=(X_test, y_test), nb_epoch=10, batch_size=64)

Train on 2515 samples, validate on 1239 samples
Epoch 1/10
2515/2515 [==============================] - 7s - loss: 1.7221 - acc: 0.3189 - val_loss: 1.2862 - val_acc: 0.3793
Epoch 2/10
2515/2515 [==============================] - 7s - loss: 1.3543 - acc: 0.4406 - val_loss: 1.0113 - val_acc: 0.5165
Epoch 3/10
2515/2515 [==============================] - 7s - loss: 1.1396 - acc: 0.4855 - val_loss: 0.9914 - val_acc: 0.4964
Epoch 4/10
2515/2515 [==============================] - 7s - loss: 1.0243 - acc: 0.5133 - val_loss: 0.9237 - val_acc: 0.5504
Epoch 5/10
2515/2515 [==============================] - 7s - loss: 0.9722 - acc: 0.5388 - val_loss: 0.9308 - val_acc: 0.5827
Epoch 6/10
2515/2515 [==============================] - 7s - loss: 0.8951 - acc: 0.5789 - val_loss: 0.9250 - val_acc: 0.5577
Epoch 7/10
2515/2515 [==============================] - 7s - loss: 0.8096 - acc: 0.6278 - val_loss: 0.9105 - val_acc: 0.6061
Epoch 8/10
2515/2515 [==============================] - 7s - loss: 0.7445 - a

In [55]:
batch_model.layers[0].trainable=True

In [56]:
batch_model.optimizer.lr=1e-4

In [57]:
batch_model.fit(X_train, y_train, validation_data=(X_test, y_test), nb_epoch=10, batch_size=64)

Train on 2515 samples, validate on 1239 samples
Epoch 1/10
2515/2515 [==============================] - 7s - loss: 0.6056 - acc: 0.7427 - val_loss: 0.8270 - val_acc: 0.6885
Epoch 2/10
2515/2515 [==============================] - 7s - loss: 0.5679 - acc: 0.7674 - val_loss: 0.7731 - val_acc: 0.6860
Epoch 3/10
2515/2515 [==============================] - 7s - loss: 0.5519 - acc: 0.7769 - val_loss: 0.7709 - val_acc: 0.7006
Epoch 4/10
2515/2515 [==============================] - 7s - loss: 0.5315 - acc: 0.7753 - val_loss: 0.7139 - val_acc: 0.7054
Epoch 5/10
2515/2515 [==============================] - 7s - loss: 0.4923 - acc: 0.7932 - val_loss: 0.7236 - val_acc: 0.7215
Epoch 6/10
2515/2515 [==============================] - 7s - loss: 0.4918 - acc: 0.8111 - val_loss: 0.6888 - val_acc: 0.7103
Epoch 7/10
2515/2515 [==============================] - 7s - loss: 0.4599 - acc: 0.8147 - val_loss: 0.6851 - val_acc: 0.7345
Epoch 8/10
2515/2515 [==============================] - 7s - loss: 0.4082 - a

In [58]:
batch_model.layers[0].trainable=False

In [59]:
batch_model.optimizer.lr=1e-2

In [60]:
batch_model.fit(X_train, y_train, validation_data=(X_test, y_test), nb_epoch=5, batch_size=64)

Train on 2515 samples, validate on 1239 samples
Epoch 1/5
2515/2515 [==============================] - 7s - loss: 0.3669 - acc: 0.8604 - val_loss: 0.6471 - val_acc: 0.7296
Epoch 2/5
2515/2515 [==============================] - 7s - loss: 0.3466 - acc: 0.8569 - val_loss: 0.6595 - val_acc: 0.7272
Epoch 3/5
2515/2515 [==============================] - 7s - loss: 0.3389 - acc: 0.8740 - val_loss: 0.6757 - val_acc: 0.7167
Epoch 4/5
2515/2515 [==============================] - 7s - loss: 0.3087 - acc: 0.8859 - val_loss: 0.6360 - val_acc: 0.7393
Epoch 5/5
2515/2515 [==============================] - 7s - loss: 0.2990 - acc: 0.8867 - val_loss: 0.6459 - val_acc: 0.7256


Nada.

## Pre-trained vectors III

Let's try the model from `pretrained_word_embeddings.py (+Dropout)`

In [61]:
deep_model = Sequential([
    Embedding(vocab_size, 100, input_length=seq_len, weights=[emb], trainable=False),
    Dropout(0.25),
    Convolution1D(128, 5, activation='relu'),
    Dropout(0.25),
    MaxPooling1D(5),
    Convolution1D(128, 5, activation='relu'),
    Dropout(0.25),
    MaxPooling1D(5),
    Convolution1D(128, 5, activation='relu'),
    Dropout(0.25),
    MaxPooling1D(5),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(newsgroups.target_names), activation='softmax')
])

In [62]:
deep_model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])
deep_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_5 (Embedding)          (None, 1500, 100)     0           embedding_input_5[0][0]          
____________________________________________________________________________________________________
dropout_11 (Dropout)             (None, 1500, 100)     0           embedding_5[0][0]                
____________________________________________________________________________________________________
convolution1d_4 (Convolution1D)  (None, 1496, 128)     64128       dropout_11[0][0]                 
____________________________________________________________________________________________________
dropout_12 (Dropout)             (None, 1496, 128)     0           convolution1d_4[0][0]            
___________________________________________________________________________________________

In [63]:
deep_model.fit(X_train, y_train, validation_data=(X_test, y_test), nb_epoch=10, batch_size=64)

Train on 2515 samples, validate on 1239 samples
Epoch 1/10
2515/2515 [==============================] - 7s - loss: 1.2744 - acc: 0.3825 - val_loss: 1.0677 - val_acc: 0.4826
Epoch 2/10
2515/2515 [==============================] - 7s - loss: 0.9629 - acc: 0.4970 - val_loss: 0.9087 - val_acc: 0.5214
Epoch 3/10
2515/2515 [==============================] - 7s - loss: 0.8403 - acc: 0.5372 - val_loss: 0.8243 - val_acc: 0.5609
Epoch 4/10
2515/2515 [==============================] - 7s - loss: 0.8032 - acc: 0.5539 - val_loss: 0.8088 - val_acc: 0.5666
Epoch 5/10
2515/2515 [==============================] - 7s - loss: 0.8051 - acc: 0.5463 - val_loss: 0.7987 - val_acc: 0.5609
Epoch 6/10
2515/2515 [==============================] - 7s - loss: 0.7344 - acc: 0.5877 - val_loss: 0.7587 - val_acc: 0.5932
Epoch 7/10
2515/2515 [==============================] - 7s - loss: 0.7298 - acc: 0.6072 - val_loss: 0.7457 - val_acc: 0.6094
Epoch 8/10
2515/2515 [==============================] - 7s - loss: 0.6880 - a

In [64]:
deep_model.layers[0].trainable=True

In [65]:
deep_model.optimizer.lr=1e-4

In [66]:
deep_model.fit(X_train, y_train, validation_data=(X_test, y_test), nb_epoch=10, batch_size=64)

Train on 2515 samples, validate on 1239 samples
Epoch 1/10
2515/2515 [==============================] - 7s - loss: 0.5615 - acc: 0.7443 - val_loss: 0.6742 - val_acc: 0.7175
Epoch 2/10
2515/2515 [==============================] - 7s - loss: 0.5248 - acc: 0.7734 - val_loss: 0.5936 - val_acc: 0.7554
Epoch 3/10
2515/2515 [==============================] - 7s - loss: 0.4579 - acc: 0.8024 - val_loss: 0.5693 - val_acc: 0.7587
Epoch 4/10
2515/2515 [==============================] - 7s - loss: 0.4303 - acc: 0.8223 - val_loss: 0.5713 - val_acc: 0.7312
Epoch 5/10
2515/2515 [==============================] - 7s - loss: 0.4229 - acc: 0.8147 - val_loss: 0.5951 - val_acc: 0.7490
Epoch 6/10
2515/2515 [==============================] - 7s - loss: 0.4053 - acc: 0.8278 - val_loss: 0.5761 - val_acc: 0.7385
Epoch 7/10
2515/2515 [==============================] - 7s - loss: 0.3846 - acc: 0.8366 - val_loss: 0.5642 - val_acc: 0.7708
Epoch 8/10
2515/2515 [==============================] - 7s - loss: 0.3352 - a

In [67]:
deep_model.optimizer.lr=1e-3

In [68]:
deep_model.fit(X_train, y_train, validation_data=(X_test, y_test), nb_epoch=10, batch_size=64)

Train on 2515 samples, validate on 1239 samples
Epoch 1/10
2515/2515 [==============================] - 7s - loss: 0.3149 - acc: 0.8700 - val_loss: 0.5699 - val_acc: 0.7619
Epoch 2/10
2515/2515 [==============================] - 7s - loss: 0.2921 - acc: 0.8815 - val_loss: 0.6360 - val_acc: 0.7498
Epoch 3/10
2515/2515 [==============================] - 7s - loss: 0.3134 - acc: 0.8807 - val_loss: 0.5673 - val_acc: 0.7708
Epoch 4/10
2515/2515 [==============================] - 7s - loss: 0.2430 - acc: 0.9014 - val_loss: 0.5944 - val_acc: 0.7635
Epoch 5/10
2515/2515 [==============================] - 7s - loss: 0.2447 - acc: 0.9010 - val_loss: 0.5684 - val_acc: 0.7789
Epoch 6/10
2515/2515 [==============================] - 7s - loss: 0.2369 - acc: 0.9014 - val_loss: 0.5961 - val_acc: 0.7789
Epoch 7/10
2515/2515 [==============================] - 7s - loss: 0.2092 - acc: 0.9205 - val_loss: 0.5877 - val_acc: 0.7684
Epoch 8/10
2515/2515 [==============================] - 7s - loss: 0.1836 - a

Better than the first two pretrained embedding models and back to being competitive with the single conv layer with max pooling.

## Multi-size CNN

This is an implementation of a multi-size CNN as shown in Ben Bowles' [excellent blog post](https://quid.com/feed/how-quid-uses-deep-learning-with-small-data).

In [69]:
from keras.layers import Merge

We use the functional API to create multiple conv layers of different sizes, and then concatenate them.

In [70]:
from keras.layers import Input, Merge
from keras.models import Model

graph_in = Input((vocab_size, 100))
convs = [] 
for fsz in range (3, 6): 
    x = Convolution1D(64, fsz, border_mode='same', activation='relu')(graph_in)
    x = MaxPooling1D()(x) 
    x = Flatten()(x) 
    convs.append(x)
out = Merge(mode='concat')(convs) 
graph = Model(graph_in, out) 

In [71]:
emb = create_emb()

We then replace the conv/max-pool layer in our original CNN with the concatenated conv layers.

In [72]:
multi = Sequential ([
    Embedding(vocab_size, 100, input_length=seq_len, dropout=0.2, weights=[emb]),
    Dropout (0.2),
    graph,
    Dropout (0.5),
    Dense (100, activation='relu'),
    Dropout (0.7),
    Dense (len(newsgroups.target_names), activation='softmax')
    ])

In [73]:
multi.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])
multi.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_6 (Embedding)          (None, 1500, 100)     1000000     embedding_input_6[0][0]          
____________________________________________________________________________________________________
dropout_16 (Dropout)             (None, 1500, 100)     0           embedding_6[0][0]                
____________________________________________________________________________________________________
model_1 (Model)                  multiple              76992       dropout_16[0][0]                 
____________________________________________________________________________________________________
dropout_17 (Dropout)             (None, 144000)        0           model_1[1][0]                    
___________________________________________________________________________________________

In [74]:
multi.fit(X_train, y_train, validation_data=(X_test, y_test), nb_epoch=10, batch_size=64)

Train on 2515 samples, validate on 1239 samples
Epoch 1/10
2515/2515 [==============================] - 11s - loss: 1.3904 - acc: 0.3690 - val_loss: 1.1430 - val_acc: 0.4835
Epoch 2/10
2515/2515 [==============================] - 11s - loss: 1.0033 - acc: 0.4954 - val_loss: 0.8661 - val_acc: 0.5440
Epoch 3/10
2515/2515 [==============================] - 11s - loss: 0.8767 - acc: 0.5551 - val_loss: 0.7793 - val_acc: 0.6021
Epoch 4/10
2515/2515 [==============================] - 11s - loss: 0.7661 - acc: 0.6171 - val_loss: 0.6938 - val_acc: 0.6642
Epoch 5/10
2515/2515 [==============================] - 11s - loss: 0.6566 - acc: 0.7066 - val_loss: 0.6247 - val_acc: 0.7183
Epoch 6/10
2515/2515 [==============================] - 11s - loss: 0.5343 - acc: 0.7773 - val_loss: 0.5830 - val_acc: 0.7328
Epoch 7/10
2515/2515 [==============================] - 11s - loss: 0.4578 - acc: 0.8087 - val_loss: 0.5403 - val_acc: 0.7595
Epoch 8/10
2515/2515 [==============================] - 11s - loss: 0.

In [75]:
multi.layers[0].trainable=False

In [76]:
multi.optimizer.lr=1e-5

In [77]:
multi.fit(X_train, y_train, validation_data=(X_test, y_test), nb_epoch=10, batch_size=64)

Train on 2515 samples, validate on 1239 samples
Epoch 1/10
2515/2515 [==============================] - 11s - loss: 0.2732 - acc: 0.9014 - val_loss: 0.5192 - val_acc: 0.8006
Epoch 2/10
2515/2515 [==============================] - 11s - loss: 0.2558 - acc: 0.9062 - val_loss: 0.5460 - val_acc: 0.7893
Epoch 3/10
2515/2515 [==============================] - 11s - loss: 0.2126 - acc: 0.9201 - val_loss: 0.5266 - val_acc: 0.7990
Epoch 4/10
2515/2515 [==============================] - 11s - loss: 0.1891 - acc: 0.9316 - val_loss: 0.5566 - val_acc: 0.8031
Epoch 5/10
2515/2515 [==============================] - 11s - loss: 0.1983 - acc: 0.9284 - val_loss: 0.6024 - val_acc: 0.7926
Epoch 6/10
2515/2515 [==============================] - 11s - loss: 0.1702 - acc: 0.9412 - val_loss: 0.5883 - val_acc: 0.7934
Epoch 7/10
2515/2515 [==============================] - 11s - loss: 0.1496 - acc: 0.9475 - val_loss: 0.5539 - val_acc: 0.8216
Epoch 8/10
2515/2515 [==============================] - 11s - loss: 0.

In [78]:
multi.layers[0].trainable=True

In [79]:
multi.optimizer.lr=1e-2

In [80]:
multi.fit(X_train, y_train, validation_data=(X_test, y_test), nb_epoch=5, batch_size=64)

Train on 2515 samples, validate on 1239 samples
Epoch 1/5
2515/2515 [==============================] - 11s - loss: 0.1515 - acc: 0.9483 - val_loss: 0.6178 - val_acc: 0.8192
Epoch 2/5
2515/2515 [==============================] - 11s - loss: 0.1193 - acc: 0.9590 - val_loss: 0.6593 - val_acc: 0.8208
Epoch 3/5
2515/2515 [==============================] - 11s - loss: 0.1021 - acc: 0.9630 - val_loss: 0.6879 - val_acc: 0.8160
Epoch 4/5
2515/2515 [==============================] - 11s - loss: 0.1129 - acc: 0.9610 - val_loss: 0.6690 - val_acc: 0.8176
Epoch 5/5
2515/2515 [==============================] - 11s - loss: 0.1175 - acc: 0.9539 - val_loss: 0.6568 - val_acc: 0.8216


Highest deep-learning result so far! And the most comparable to the 'shallow' bag-of-words results that achieved upper-80s.

Bonus: this was also the least stressful to watch train because `val_acc` (generally) continued to rise instead of bouncing around like the other models.